# Automated ML

In the cell below, we import all the dependencies that we will need to complete the project.

In [40]:
from azureml.core import Workspace, Dataset, ComputeTarget
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.automl.runtime.onnx_convert import OnnxConverter

## Dataset

### Overview

For our capstone project, we use [the kaggle heart failure dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). We uploaded and registered this dataset to the workspace beforehand. The dataset stems from [a publication on heart failure prediction using machine learning](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5). The dataset contains medical records for 299 patients with heart failures as well as their survival as a binary variable ("DEATH_EVENT"). The goal of our AutoML - experiment is to predict survival. Hence, we are dealing with a binary classification problem at the core of the task.

In [41]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-experiment'
experiment=Experiment(ws, experiment_name)

# print some information about the experiment
print(f'Workspace name: {ws.name} / AZ region: {ws.location} ' \
    f'/ Subscription ID: {ws.subscription_id} / Resource group: {ws.resource_group}')

# start logging and get the compute target
run = experiment.start_logging()
cluster_name = "capstone-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,
        min_node_count=None,
        timeout_in_minutes=60,
    )

# message if ready
print(f'compute target: {compute_target.get_status().serialize()}')

2023-08-08:18:32:27,951 INFO     [workspace.py:291] Found the config file in: /config.json


Workspace name: quick-starts-ws-239936 / AZ region: westus2 / Subscription ID: 9b72f9e6-56c5-4c16-991b-19c652994860 / Resource group: aml-quickstarts-239936
Found existing compute target: AmlCompute(workspace=Workspace.create(name='quick-starts-ws-239936', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-239936'), name=capstone-cluster, id=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourceGroups/aml-quickstarts-239936/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-239936/computes/capstone-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westus2, tags={})
compute target: {'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-08-08T17:46:06.044000+00:00', 'errors': None, 'creationTime

In [42]:
ds_name = 'heart_failure_kaggle_ml'
dataset = Dataset.get_by_name(workspace=ws, name=ds_name)

In [43]:
# inspect the dataframe 
pd_dataset = dataset.to_pandas_dataframe()
pd_dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


In [44]:
pd_dataset.tail()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
294,62.00,0,61,1,38,1,155000.00,1.10,143,1,1,270,0
295,55.00,0,1820,0,38,0,270000.00,1.20,139,0,0,271,0
296,45.00,0,2060,1,60,0,742000.00,0.80,138,0,0,278,0
297,45.00,0,2413,0,38,0,140000.00,1.40,140,1,1,280,0
298,50.00,0,196,0,45,0,395000.00,1.60,136,1,1,285,0


## AutoML Configuration

The AutoML configuration below considers the following aspects:
- the task, which is set to classification for our usecase
- the primary metric used to evaluate the metrics - here we choose accuracy
- the training data, which is simply the Azure data asset described above
- the name of the target variable column in the dataset: DEATH_EVENT
- the number of cross validation folds used to train and evaluate the models to get a better overview of the individual model's performance
- the compute target to run our AutoML job as described above
- a flag to enable early stopping so that the experiment ends if the results do not improve in any way
- the experiment timeout minutes - we set to 20 minutes to be able to react fast if the experiment goes wrong
- we allow a max. number of 5 concurrent iterations to not overload our compute
- we set up a path for storing the experiment results: ./automl-rum
- we set the flag on featurization on "auto" to be able to profit from various AutomL-techniques for data (pre-) processing
- and finally, we set up a name for the AutoML - logging: automl_errors.log

In [45]:
target_column = "DEATH_EVENT"

# automl experiment settings here
automl_settings = {
    "task": "classification",
    "primary_metric": "accuracy",
    "training_data": dataset,
    "label_column_name": target_column,
    "n_cross_validations": 5,
    "compute_target": compute_target,
    "enable_early_stopping": True,
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "path": "./automl-run",
    "featurization": "auto",
    "debug_log": "automl_errors.log"
}

# automl config here
automl_config = AutoMLConfig(**automl_settings)

In [46]:
# submit the experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-experiment,AutoML_56298064-a8a3-4016-9d94-b5507dfcf65c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In the cell below, we use the `RunDetails` widget to show the different experiments.

In [47]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-experiment,AutoML_56298064-a8a3-4016-9d94-b5507dfcf65c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_56298064-a8a3-4016-9d94-b5507dfcf65c',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-08-08T18:32:57.097764Z',
 'endTimeUtc': '2023-08-08T18:42:54.235433Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-experiment","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-239936","workspace_name":"quick-starts-ws-239936","region":"westus2","compute_target":"capston

## Best Model

In the following cells, we display some information about the AutoML run, the best run metrics, get the best model from the automl experiments and display all the properties of the model.

In [48]:
best_run, best_model = automl_run.get_output()
print(f"ID of best run: {best_run.id}")

2023-08-08:18:43:16,402 WARNING  [package_utilities.py:203] The version of the SDK does not match the version the model was trained on.
2023-08-08:18:43:16,403 WARNING  [package_utilities.py:204] The consistency in the result may not be guaranteed.
2023-08-08:18:43:16,404 WARNING  [package_utilities.py:209] Package:azureml-automl-core, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, curren

ID of best run: AutoML_56298064-a8a3-4016-9d94-b5507dfcf65c_51


In [49]:
print(f"Metrics results of the best run: {best_run.get_metrics()}")

Metrics results of the best run: {'f1_score_micro': 0.8897175141242938, 'f1_score_macro': 0.8660905548676316, 'balanced_accuracy': 0.8619047619047621, 'precision_score_weighted': 0.9048272317148681, 'weighted_accuracy': 0.908408072959493, 'recall_score_micro': 0.8897175141242938, 'average_precision_score_micro': 0.9194012493742687, 'norm_macro_recall': 0.7238095238095238, 'f1_score_weighted': 0.8859539915484156, 'accuracy': 0.8897175141242938, 'recall_score_weighted': 0.8897175141242938, 'precision_score_micro': 0.8897175141242938, 'AUC_micro': 0.9174876951067701, 'log_loss': 0.36687628265694283, 'AUC_macro': 0.9164768595422667, 'average_precision_score_weighted': 0.9285189629614254, 'matthews_correlation': 0.7553025119722379, 'recall_score_macro': 0.8619047619047621, 'AUC_weighted': 0.9164768595422667, 'precision_score_macro': 0.8961048938444603, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_56298064-a8a3-4016-9d94-b5507dfcf65c_51/accuracy_table', 'average_precision_sc

In [50]:
print(f"Overview over the best model and its details: {best_model}")

Overview over the best model and its details: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('32', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=1, colsample_bytree=1, eta=0.5, gamma=0.01, max_depth=7, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.5208333333333334, reg_lambda=2.291666666666667, subsample=1, 

In [51]:
# save the best model
best_run = automl_run.get_best_child()
best_model_name = best_run.properties["model_name"]
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'logs/azureml/azureml_automl.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/engineered_feature_names.json',
 'outputs/env_dependencies.json',
 'outputs/featurization_summary.json',
 'outputs/generated_code/script.py',
 'outputs/generated_code/script_run_notebook.ipynb',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/run_id.txt',
 'outputs/scoring_file_pbi_v_1_0_0.py',
 'outputs/scoring_file_v_1_0_0.py',
 'outputs/scoring_file_v_2_0_0.py',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [52]:
best_run.download_files("./outputs")  # model.pkl is now saved in ./outputs

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

As the models are roughly similar, we chose to just register the AutoML model here.

In [53]:
best_model_name = best_run.properties["model_name"]
model = best_run.register_model(
    model_name=best_model_name, 
    model_path="./outputs"
)

In [54]:
print(f"Best run model name is : {best_model_name}")

Best run model name is : AutoML56298064a51


2023-08-08:19:23:14,749 WARNING  [connectionpool.py:823] Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fdb8ab31dc0>, 'Connection to westus2.experiments.azureml.net timed out. (connect timeout=100)')': /history/v1.0/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourceGroups/aml-quickstarts-239936/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-239936/experimentids/8622493b-ce67-4e10-b7b8-b298dcf8a301/runs/AutoML_90d2b2e5-9748-4e7e-aabc-94997ceff017
2023-08-08:19:24:01,847 WARNING  [connectionpool.py:823] Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fdb8ae1ae50>, 'Connection to westus2.experiments.azureml.net timed out. (connect timeout=100)')': /history/v1.0/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/reso

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
